In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import optuna
import datetime
import joblib

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
color_pal = sns.color_palette()
pd.options.plotting.backend = "plotly"
plt.style.use('fivethirtyeight')

c:\Users\robinson.beaucour\PythonProjects\Machine Learning\MLenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load prediction in France

### Load data

In [2]:
Load = pd.read_csv("./data_input/Load.csv",sep=';',usecols=['FR','DateTime'])
Load.index = pd.to_datetime(Load['DateTime'],format='%d/%m/%Y %H:%M:%S.%f').dt.round('H')
Load.drop(['DateTime'],axis=1,inplace=True)
Load.columns = ['Load']

### Features

In [3]:
# Temperature
TA = pd.read_csv("./data_input/TA_tab.csv")
TA.index = pd.to_datetime(TA[['Year','Month','Day']])
TA.drop(['Year','Month','Day'],axis=1,inplace=True)
# Public holiday
Raw_JF = pd.read_csv('./data_input/jours_feries_metropole.csv')
Raw_JF.index = pd.to_datetime(Raw_JF['date'])
JF = TA[[]].copy()
JF['Férié'] = 0
JF.loc[:,'Férié'].loc[Raw_JF.loc[(Raw_JF.index>='2014')&(Raw_JF.index<'2020-01-01')].index] = 1
# Resample from day to hours
TA = TA.resample('h').ffill()
JF = JF.resample('h').ffill()

Temperatures that are the most negatively correlated with load in France between 2015 and 2020.

In [4]:
correlations = pd.concat(
                [Load.query('index>="2015" & index<"2020"'),
                TA.query('index>="2015" & index<"2020"')],axis=1).corr()
correlations[['Load']].nsmallest(10,columns='Load')

,Load
ITC4,-0.789307
ITH2,-0.789079
ITH1,-0.787501
ITH4,-0.787022
DE27,-0.786647
ITC1,-0.786626
AT21,-0.786171
CH05,-0.786077
ITH3,-0.786007
CH07,-0.785964


In [5]:
from utils import temperature_pca, create_time_features

Features creation

In [6]:
# Features initialisation
Features = pd.DataFrame(index=Load.index)
# Time features
Features = Features.merge(
    create_time_features(Load),
    how='left',
    left_index=True,
    right_index=True
    )
# Temperature PCA on all location features
Features = Features.merge(
    temperature_pca(TA,0.95,'all'),
    how='left',
    left_index=True,
    right_index=True
    )
# Temperature PCA on France features
Features = Features.merge(
    temperature_pca(TA,0.95,'FR'),
    how='left',
    left_index=True,
    right_index=True
    )
# Public Holiday features
Features = Features.merge(
    JF,
    how='left',
    left_index=True,
    right_index=True
    )

### Train & Test sets

In [7]:
X_train = Features.query('index >= 2015&index < 2019')
y_train = Load.query('index >= 2015&index < 2019')['Load']

X_test = Features.query('index >= 2019&index < 2020')
y_test = Load.query('index >= 2019&index < 2020')['Load']

In [8]:
reg = XGBRegressor(base_score=0.5, booster='gbtree',    
                       n_estimators=1500,
                       early_stopping_rounds=50,
                       objective='reg:linear',
                       max_depth=15,
                       learning_rate=0.01)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=100)

[23:22:00] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-030221e36e1a46bfb-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:54785.63998	validation_1-rmse:54011.51782
[100]	validation_0-rmse:20264.48971	validation_1-rmse:19991.74237
[200]	validation_0-rmse:7631.74434	validation_1-rmse:7764.75731
[300]	validation_0-rmse:2985.77713	validation_1-rmse:3741.53700
[400]	validation_0-rmse:1247.30189	validation_1-rmse:2736.10674
[500]	validation_0-rmse:579.60399	validation_1-rmse:2540.75009
[600]	validation_0-rmse:315.18747	validation_1-rmse:2496.01477
[700]	validation_0-rmse:203.21624	validation_1-rmse:2484.04711
[800]	validation_0-rmse:152.58878	validation_1-rmse:2479.86257
[900]	validation_0-rmse:123.62516	validation_1-rmse:2477.66065
[1000]	validation_0-rmse:106.25528	validation_1-rmse:2476.31169
[1100]	validation_0-rmse:89.01042	validation_1-rmse:2475.34484
[1200

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=50, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=15, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1500, n_jobs=0,
             num_parallel_tree=1, objective='reg:linear', predictor='auto', ...)

### Optuna

In [9]:
study = optuna.create_study(direction='minimize', study_name="starter-experiment")

[I 2022-12-25 23:22:49,216] A new study created in memory with name: starter-experiment


In [10]:
def objective(trial):
    params = {
        'learning_rate' : trial.suggest_float('learning_rate', 1e-5, 1e-2),
        'n_estimators' : trial.suggest_int('n_estimators', 700, 2000),
        'max_depth' : trial.suggest_int('max_depth', 3, 20),
        'importance_type' : trial.suggest_categorical('importance_type', ['weight', 'gain', 'cover']),
        'reg_lambda' : trial.suggest_float('lambda', 1e-8, 1.0),
        'reg_alpha' : trial.suggest_float('alpha', 1e-8, 1.0)
    }

    # Create an XGBRegressor model with the sampled hyperparameters
    model = XGBRegressor(**params)
    
    # Fit the model on the training data
    model.fit(X_train, y_train)
    
    # Calculate the mean squared error on the validation set
    mse = mean_squared_error(y_test, model.predict(X_test))
    
    # Return the negative MSE to minimize the objective
    return mse


In [11]:
# Get the current date and time
now = datetime.datetime.now()
# Format the current date and time as a string without the seconds
now_str = now.strftime("%Y-%m-%d %H_%M")
n_trials = 300
study = optuna.create_study(direction='minimize',study_name=now_str+f'--{n_trials}')
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2022-12-25 23:22:49,361] A new study created in memory with name: 2022-12-25 23_22--300
c:\Users\robinson.beaucour\PythonProjects\Machine Learning\MLenv\lib\site-packages\optuna\progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()
  0%|          | 1/300 [00:28<2:23:09, 28.73s/it]

[I 2022-12-25 23:23:18,101] Trial 0 finished with value: 499407667.8899881 and parameters: {'learning_rate': 0.0005339216168557866, 'n_estimators': 1684, 'max_depth': 20, 'importance_type': 'weight', 'lambda': 0.3824543118313093, 'alpha': 0.833292545372051}. Best is trial 0 with value: 499407667.8899881.


  1%|          | 2/300 [00:33<1:14:07, 14.93s/it]

[I 2022-12-25 23:23:23,364] Trial 1 finished with value: 12055966.574690487 and parameters: {'learning_rate': 0.00329880148209808, 'n_estimators': 1418, 'max_depth': 3, 'importance_type': 'weight', 'lambda': 0.7063674068089868, 'alpha': 0.5968336537471658}. Best is trial 1 with value: 12055966.574690487.


  1%|          | 3/300 [00:48<1:13:19, 14.81s/it]

...

[I 2022-12-26 01:53:48,041] Trial 297 finished with value: 5882681.625218832 and parameters: {'learning_rate': 0.007035739904081659, 'n_estimators': 1628, 'max_depth': 11, 'importance_type': 'cover', 'lambda': 0.47635892641390243, 'alpha': 0.01760938339797674}. Best is trial 208 with value: 5671461.643133052.


100%|█████████▉| 299/300 [2:31:29<00:31, 31.07s/it]

[I 2022-12-26 01:54:19,309] Trial 298 finished with value: 5727408.0279764235 and parameters: {'learning_rate': 0.007778472119717058, 'n_estimators': 1881, 'max_depth': 10, 'importance_type': 'cover', 'lambda': 0.4993856426590607, 'alpha': 0.00028762677924164953}. Best is trial 208 with value: 5671461.643133052.


100%|██████████| 300/300 [2:31:56<00:00, 30.39s/it]

[I 2022-12-26 01:54:46,246] Trial 299 finished with value: 5866601.456805089 and parameters: {'learning_rate': 0.007671509563052032, 'n_estimators': 1926, 'max_depth': 9, 'importance_type': 'cover', 'lambda': 0.4439890289422263, 'alpha': 0.05069725058117307}. Best is trial 208 with value: 5671461.643133052.


In [12]:
best_params = study.best_params
model = XGBRegressor(**best_params)
model.fit(X_train,y_train)

# Evaluate the model on the test set
test_mse = mean_squared_error(y_test, model.predict(X_test))
print(f'Test MSE: {test_mse:.3f}')
print(f'Test RMSE: {np.sqrt(test_mse):.3f}')

Test MSE: 5671461.643
Test RMSE: 2381.483


In [14]:
from utils import features_importance
features_importance(model)

In [16]:
y_predict = model.predict(X_test)
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = Load.index,
        y = Load.Load,
        name = 'Truth Data'
    )
)
fig.add_trace(
    go.Scatter(
        x = X_test.index,
        y = y_predict,
        name = 'Prediction'
    )
)
fig.update_layout(
    title = f"RMSE for 2019 forecast : {np.round(np.sqrt(mean_squared_error(y_test,y_predict)),2)}",
    hovermode='x'
)

In [17]:
fig = optuna.visualization.plot_optimization_history(study)
fig.update_layout(hovermode='x')
fig.update_yaxes(type="log")
fig.show()

In [18]:
optuna.visualization.plot_param_importances(study)

In [19]:
fig = optuna.visualization.plot_slice(study)
fig.update_yaxes(type="log")
fig.show()


In [20]:
# Get the current date and time
now = datetime.datetime.now()
# Format the current date and time as a string without the seconds
now_str = now.strftime("%Y-%m-%d %H_%M")
joblib.dump(study, f"./OPTUNA_results/{now_str}--{n_trials}.pkl")

['./OPTUNA_results/2022-12-26 11_17--300.pkl']